# RAG z uporabo Azure AI Agent Service in Semantic Kernel

Ta del kode prikazuje, kako ustvariti in upravljati Azure AI agenta za pridobivanje z izboljšano generacijo (RAG) z uporabo `Azure AI Agent Service` in `Semantic Kernel`. Agent obdeluje uporabniška vprašanja na podlagi pridobljenega konteksta in ustrezno zagotavlja natančne odgovore.


Popravek različice SQLite  
Če naletite na napako:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Odkomentirajte ta blok kode na začetku vašega zvezka:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Uvažanje paketov
Naslednja koda uvozi potrebne pakete:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Pridobivanje z dodatno generacijo s Semantic Kernel in Azure AI Agent Service

Ta primer prikazuje, kako uporabiti **Azure AI Agent Service** za izvajanje **pridobivanja z dodatno generacijo (RAG)**, tako da združite jezikovni model z vsebinskim kontekstom iz naloženega dokumenta.

### Kako deluje

1. **Nalaganje dokumenta**: Markdown datoteka (document.md), ki vsebuje informacije (Contosova politika potovalnega zavarovanja), se naloži v storitev agenta.

2. **Ustvarjanje vektorske shrambe**: Dokument se indeksira v vektorsko shrambo, da omogoči semantično iskanje po njegovi vsebini.

3. **Konfiguracija agenta**: Agent se ustvari z modelom `gpt-4o` in naslednjimi strogimi navodili:
   - Odgovarjaj na vprašanja samo na podlagi pridobljene vsebine iz dokumenta.
   - Zavrnite odgovor, če je vprašanje izven obsega.

4. **Integracija orodja za iskanje datotek**: `FileSearchTool` je registrirano pri agentu, kar omogoča modelu iskanje in pridobivanje ustreznih odlomkov iz indeksiranega dokumenta med sklepanjem.

5. **Interakcija z uporabnikom**: Uporabniki lahko postavljajo vprašanja. Če so v dokumentu najdene ustrezne informacije, agent ustvari utemeljen odgovor.  
   Če ne, agent izrecno odgovori, da dokument ne vsebuje dovolj informacij.


### Glavna funkcija



Prepričajte se, da najprej zaženete `az login` z uporabo Azure CLI, da zagotovite ustrezen kontekst za preverjanje pristnosti med uporabo `DefaultAzureCredential`. Storitev Azure AI Agent Service ne uporablja ključev API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Seveda! Prosim, prilepite vsebino datoteke, ki jo želite prevesti, in začel bom s prevajanjem.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni prevod s strani človeka. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
